In [ ]:
from kaggle_environments import make, evaluate

# Create the game environment
# Set debug=True to see the errors if your agent refuses to run
env = make("connectx", debug=True)

# The approach

I have experimented a lot and  it's my best agent. The main points of the solution:  

* **Bitboards** to store the board position. Links: [Wikipedia](https://en.wikipedia.org/wiki/Bitboard), [Chessprogramming wiki](https://www.chessprogramming.org/Bitboards). It makes the agent
much faster the search deeper.

* **Alpha-Beta pruning** [Wikipedia](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning).

* **Iterative deepening** [Wikipedia](https://en.wikipedia.org/wiki/Iterative_deepening_depth-first_search).

* **Cache** - I don't name it [Transposition table](https://en.wikipedia.org/wiki/Negamax#Negamax_with_alpha_beta_pruning_and_transposition_tables), it is
mush simplier, but it seems to work in this case.

### The heuristic

It is rather common, the main part calcs numbers of threes and twos, but with original function (State.count23) using bitboards, much faster.
Plus the heurictic calcs number of player's pieces in the even and odd rows.

Then the heuristic checks if the player or opponent has "doublewin" position, i.e. unstoppable win, see the State's *doublewin* method below.

*The time I write this the uncommented draft version of the agent is in the top 5% (11th, score 1330) of the Leaderboard*

**Thanks** [Gilles Vandewiele](https://www.kaggle.com/group16) and [Taaha Khan](https://www.kaggle.com/taahakhan) for ideas!

#### Please don't forget to upvote if you like it or use some parts, thanks.

In [ ]:
def my_agent(obs, config):
    import numpy as np
    import random
    import time

    # Precalculate some constants for better performance
    W = config.inarow
    R = config.rows
    C = config.columns
    Rp1 = R + 1
    Rm1 = R - 1
    RmW = R - W
    RmWp1 = R - W + 1
    Cp1 = C + 1
    Cm1 = C - 1
    Rp1Cp1 = Rp1 * Cp1
    Ct2 = C * 2
    Cp1t2 = Cp1 * 2
    Cm1t2 = Cm1 * 2
    Wmask = 2**W - 1
    RC = R*C
    DELAY = 4.98
                
    # the order of calculation the scores of a move, nearest to the center first, 
    # in the case of 7 columns it's [3, 2, 4, 1, 5, 0, 6]
    ORDER = [C//2 - i//2 - 1 if i%2 else C//2 + i//2 for i in range(C)]
    
    # Some utility functions
    
    def step_number(grid):
        # returns step of the game
        empty = 0
        for i in range(R):
            for j in range(C):
                if grid[i][j] == 0:
                    empty += 1
        return R*C - empty

    def is_first_player(step):
        return step%2 == 0
    
    def int_to_bin(x):
        # 64-bit integer to 64-letter string
        return bin(x)[2:].zfill(64)
    
    def grid_to_matrix(grid):
        matrix = np.array([0 for i in range(Rp1Cp1)]).reshape(Rp1, Cp1)
        for r in range(R):
            for c in range(C):
                matrix[r+1, c] = grid[r, c]
        return matrix

    def grid_to_state(grid):
        matrix = grid_to_matrix(grid)
        position1, position2 = '', ''
        for c in range(C, -1, -1):
            for r in range(0, Rp1):
                if matrix[r,c] == 1:
                    position1 += '1'
                else:
                    position1 += '0'
                if matrix[r,c] == 2:
                    position2 += '1'
                else:
                    position2 += '0'
        return {1:int(position1, 2), 2:int(position2, 2)}

    def state_to_grid(state):
        position1 = state[1]
        position2 = state[2]
        matrix = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0]])
        for c in range(0, Cp1):
            for r in range(R, -1, -1):
                if position1 & 1:
                    matrix[r,c] = 1
                position1 = position1 >> 1
                if position2 & 1:
                    matrix[r,c] = 2
                position2 = position2 >> 1
        return matrix[1:, :-1]

    def make_move(state, col, mark):
        mask = state[1] | state[2]
        new_mask = mask | (mask + (1 << (col*7)))
        new_state = state.copy()
        new_state[mark] = state[3-mark] ^ new_mask
        return new_state

    def first_empty_row(state, col):
        m = state[1] | state[2]
        for row in range(6, -1, -1):
            if not (m >> 7*col+6-row) & 1:
                return 6-row

    def connected_four(position):
        # Horizontal check
        m = position & (position >> 7)
        if m & (m >> 14): return True
        # Diagonal \
        m = position & (position >> 6)
        if m & (m >> 12): return True
        # Diagonal /
        m = position & (position >> 8)
        if m & (m >> 16): return True
        # Vertical
        m = position & (position >> 1)
        if m & (m >> 2): return True
        # Nothing found
        return False

    def vertical_four(position):
        m = position & (position >> 1)
        if m & (m >> 2): 
            return True
        return False

    def int_to_bin(x):
        return bin(x)[2:].zfill(64)

    def valid_moves_list(state):
        m = state[1] | state[2]
        vm = [i for i in [3,2,4,1,5,0,6] if not (m >> 5+7*i) & 1] 
        return vm
   
    MASKS = [{4: 60, 3: [28, 44, 52, 56], 2: [12, 20, 24, 36, 40, 48]}, {4: 30, 3: [14, 22, 26, 28], 2: [6, 10, 12, 18, 20, 24]}, {4: 15, 3: [7, 11, 13, 14], 2: [3, 5, 6, 9, 10, 12]}, {4: 7680, 3: [3584, 5632, 6656, 7168], 2: [1536, 2560, 3072, 4608, 5120, 6144]}, {4: 3840, 3: [1792, 2816, 3328, 3584], 2: [768, 1280, 1536, 2304, 2560, 3072]}, {4: 1920, 3: [896, 1408, 1664, 1792], 2: [384, 640, 768, 1152, 1280, 1536]}, {4: 983040, 3: [458752, 720896, 851968, 917504], 2: [196608, 327680, 393216, 589824, 655360, 786432]}, {4: 491520, 3: [229376, 360448, 425984, 458752], 2: [98304, 163840, 196608, 294912, 327680, 393216]}, {4: 245760, 3: [114688, 180224, 212992, 229376], 2: [49152, 81920, 98304, 147456, 163840, 196608]}, {4: 125829120, 3: [58720256, 92274688, 109051904, 117440512], 2: [25165824, 41943040, 50331648, 75497472, 83886080, 100663296]}, {4: 62914560, 3: [29360128, 46137344, 54525952, 58720256], 2: [12582912, 20971520, 25165824, 37748736, 41943040, 50331648]}, {4: 31457280, 3: [14680064, 23068672, 27262976, 29360128], 2: [6291456, 10485760, 12582912, 18874368, 20971520, 25165824]}, {4: 16106127360, 3: [7516192768, 11811160064, 13958643712, 15032385536], 2: [3221225472, 5368709120, 6442450944, 9663676416, 10737418240, 12884901888]}, {4: 8053063680, 3: [3758096384, 5905580032, 6979321856, 7516192768], 2: [1610612736, 2684354560, 3221225472, 4831838208, 5368709120, 6442450944]}, {4: 4026531840, 3: [1879048192, 2952790016, 3489660928, 3758096384], 2: [805306368, 1342177280, 1610612736, 2415919104, 2684354560, 3221225472]}, {4: 2061584302080, 3: [962072674304, 1511828488192, 1786706395136, 1924145348608], 2: [412316860416, 687194767360, 824633720832, 1236950581248, 1374389534720, 1649267441664]}, {4: 1030792151040, 3: [481036337152, 755914244096, 893353197568, 962072674304], 2: [206158430208, 343597383680, 412316860416, 618475290624, 687194767360, 824633720832]}, {4: 515396075520, 3: [240518168576, 377957122048, 446676598784, 481036337152], 2: [103079215104, 171798691840, 206158430208, 309237645312, 343597383680, 412316860416]}, {4: 263882790666240, 3: [123145302310912, 193514046488576, 228698418577408, 246290604621824], 2: [52776558133248, 87960930222080, 105553116266496, 158329674399744, 175921860444160, 211106232532992]}, {4: 131941395333120, 3: [61572651155456, 96757023244288, 114349209288704, 123145302310912], 2: [26388279066624, 43980465111040, 52776558133248, 79164837199872, 87960930222080, 105553116266496]}, {4: 65970697666560, 3: [30786325577728, 48378511622144, 57174604644352, 61572651155456], 2: [13194139533312, 21990232555520, 26388279066624, 39582418599936, 43980465111040, 52776558133248]}, {4: 67637280, 3: [67637248, 67633184, 67112992, 528416], 2: [67633152, 67112960, 528384, 67108896, 524320, 4128]}, {4: 8657571840, 3: [8657567744, 8657047552, 8590462976, 67637248], 2: [8657043456, 8590458880, 67633152, 8589938688, 67112960, 528384]}, {4: 1108169195520, 3: [1108168671232, 1108102086656, 1099579260928, 8657567744], 2: [1108101562368, 1099578736640, 8657043456, 1099512152064, 8590458880, 67633152]}, {4: 141845657026560, 3: [141845589917696, 141837067091968, 140746145398784, 1108168671232], 2: [141836999983104, 140746078289920, 1108101562368, 140737555464192, 1099578736640, 8657043456]}, {4: 33818640, 3: [33818624, 33816592, 33556496, 264208], 2: [33816576, 33556480, 264192, 33554448, 262160, 2064]}, {4: 4328785920, 3: [4328783872, 4328523776, 4295231488, 33818624], 2: [4328521728, 4295229440, 33816576, 4294969344, 33556480, 264192]}, {4: 554084597760, 3: [554084335616, 554051043328, 549789630464, 4328783872], 2: [554050781184, 549789368320, 4328521728, 549756076032, 4295229440, 33816576]}, {4: 70922828513280, 3: [70922794958848, 70918533545984, 70373072699392, 554084335616], 2: [70918499991552, 70373039144960, 554050781184, 70368777732096, 549789368320, 4328521728]}, {4: 16909320, 3: [16909312, 16908296, 16778248, 132104], 2: [16908288, 16778240, 132096, 16777224, 131080, 1032]}, {4: 2164392960, 3: [2164391936, 2164261888, 2147615744, 16909312], 2: [2164260864, 2147614720, 16908288, 2147484672, 16778240, 132096]}, {4: 277042298880, 3: [277042167808, 277025521664, 274894815232, 2164391936], 2: [277025390592, 274894684160, 2164260864, 274878038016, 2147614720, 16908288]}, {4: 35461414256640, 3: [35461397479424, 35459266772992, 35186536349696, 277042167808], 2: [35459249995776, 35186519572480, 277025390592, 35184388866048, 274894684160, 2164260864]}, {4: 8454660, 3: [8454656, 8454148, 8389124, 66052], 2: [8454144, 8389120, 66048, 8388612, 65540, 516]}, {4: 1082196480, 3: [1082195968, 1082130944, 1073807872, 8454656], 2: [1082130432, 1073807360, 8454144, 1073742336, 8389120, 66048]}, {4: 138521149440, 3: [138521083904, 138512760832, 137447407616, 1082195968], 2: [138512695296, 137447342080, 1082130432, 137439019008, 1073807360, 8454144]}, {4: 17730707128320, 3: [17730698739712, 17729633386496, 17593268174848, 138521083904], 2: [17729624997888, 17593259786240, 138512695296, 17592194433024, 137447342080, 1082130432]}, {4: 4227330, 3: [4227328, 4227074, 4194562, 33026], 2: [4227072, 4194560, 33024, 4194306, 32770, 258]}, {4: 541098240, 3: [541097984, 541065472, 536903936, 4227328], 2: [541065216, 536903680, 4227072, 536871168, 4194560, 33024]}, {4: 69260574720, 3: [69260541952, 69256380416, 68723703808, 541097984], 2: [69256347648, 68723671040, 541065216, 68719509504, 536903680, 4227072]}, {4: 8865353564160, 3: [8865349369856, 8864816693248, 8796634087424, 69260541952], 2: [8864812498944, 8796629893120, 69256347648, 8796097216512, 68723671040, 541065216]}, {4: 2113665, 3: [2113664, 2113537, 2097281, 16513], 2: [2113536, 2097280, 16512, 2097153, 16385, 129]}, {4: 270549120, 3: [270548992, 270532736, 268451968, 2113664], 2: [270532608, 268451840, 2113536, 268435584, 2097280, 16512]}, {4: 34630287360, 3: [34630270976, 34628190208, 34361851904, 270548992], 2: [34628173824, 34361835520, 270532608, 34359754752, 268451840, 2113536]}, {4: 4432676782080, 3: [4432674684928, 4432408346624, 4398317043712, 34630270976], 2: [4432406249472, 4398314946560, 34628173824, 4398048608256, 34361835520, 270532608]}, {4: 8521760, 3: [8521728, 8519712, 8390688, 133152], 2: [8519680, 8390656, 133120, 8388640, 131104, 2080]}, {4: 1090785280, 3: [1090781184, 1090523136, 1074008064, 17043456], 2: [1090519040, 1074003968, 17039360, 1073745920, 16781312, 266240]}, {4: 139620515840, 3: [139619991552, 139586961408, 137473032192, 2181562368], 2: [139586437120, 137472507904, 2181038080, 137439477760, 2148007936, 34078720]}, {4: 17871426027520, 3: [17871358918656, 17867131060224, 17596548120576, 279239983104], 2: [17867063951360, 17596481011712, 279172874240, 17592253153280, 274945015808, 4362076160]}, {4: 4260880, 3: [4260864, 4259856, 4195344, 66576], 2: [4259840, 4195328, 66560, 4194320, 65552, 1040]}, {4: 545392640, 3: [545390592, 545261568, 537004032, 8521728], 2: [545259520, 537001984, 8519680, 536872960, 8390656, 133120]}, {4: 69810257920, 3: [69809995776, 69793480704, 68736516096, 1090781184], 2: [69793218560, 68736253952, 1090519040, 68719738880, 1074003968, 17039360]}, {4: 8935713013760, 3: [8935679459328, 8933565530112, 8798274060288, 139619991552], 2: [8933531975680, 8798240505856, 139586437120, 8796126576640, 137472507904, 2181038080]}, {4: 2130440, 3: [2130432, 2129928, 2097672, 33288], 2: [2129920, 2097664, 33280, 2097160, 32776, 520]}, {4: 272696320, 3: [272695296, 272630784, 268502016, 4260864], 2: [272629760, 268500992, 4259840, 268436480, 4195328, 66560]}, {4: 34905128960, 3: [34904997888, 34896740352, 34368258048, 545390592], 2: [34896609280, 34368126976, 545259520, 34359869440, 537001984, 8519680]}, {4: 4467856506880, 3: [4467839729664, 4466782765056, 4399137030144, 69809995776], 2: [4466765987840, 4399120252928, 69793218560, 4398063288320, 68736253952, 1090519040]}, {4: 16843009, 3: [16843008, 16842753, 16777473, 65793], 2: [16842752, 16777472, 65792, 16777217, 65537, 257]}, {4: 2155905152, 3: [2155905024, 2155872384, 2147516544, 8421504], 2: [2155872256, 2147516416, 8421376, 2147483776, 8388736, 32896]}, {4: 275955859456, 3: [275955843072, 275951665152, 274882117632, 1077952512], 2: [275951648768, 274882101248, 1077936128, 274877923328, 1073758208, 4210688]}, {4: 35322350010368, 3: [35322347913216, 35321813139456, 35184911056896, 137977921536], 2: [35321811042304, 35184908959744, 137975824384, 35184374185984, 137441050624, 538968064]}, {4: 33686018, 3: [33686016, 33685506, 33554946, 131586], 2: [33685504, 33554944, 131584, 33554434, 131074, 514]}, {4: 4311810304, 3: [4311810048, 4311744768, 4295033088, 16843008], 2: [4311744512, 4295032832, 16842752, 4294967552, 16777472, 65792]}, {4: 551911718912, 3: [551911686144, 551903330304, 549764235264, 2155905024], 2: [551903297536, 549764202496, 2155872256, 549755846656, 2147516416, 8421376]}, {4: 70644700020736, 3: [70644695826432, 70643626278912, 70369822113792, 275955843072], 2: [70643622084608, 70369817919488, 275951648768, 70368748371968, 274882101248, 1077936128]}, {4: 67372036, 3: [67372032, 67371012, 67109892, 263172], 2: [67371008, 67109888, 263168, 67108868, 262148, 1028]}, {4: 8623620608, 3: [8623620096, 8623489536, 8590066176, 33686016], 2: [8623489024, 8590065664, 33685504, 8589935104, 33554944, 131584]}, {4: 1103823437824, 3: [1103823372288, 1103806660608, 1099528470528, 4311810048], 2: [1103806595072, 1099528404992, 4311744512, 1099511693312, 4295032832, 16842752]}, {4: 141289400041472, 3: [141289391652864, 141287252557824, 140739644227584, 551911686144], 2: [141287244169216, 140739635838976, 551903297536, 140737496743936, 549764202496, 2155872256]}]
    def num23(state, mark):
        n12, n13, n22, n23 = 0, 0, 0, 0
        s1, s2 = state[mark], state[3-mark]
        for m in MASKS:
            w1 = s1 & m[4]
            w2 = s2 & m[4]
            if not w2 and w1:
                for m3 in m[3]:
                    if w1 == m3:
                        n13 += 1
                for m2 in m[2]:
                    if w1 == m2:
                        n12 += 1
            if not w1 and w2:
                for m3 in m[3]:
                    if w2 == m3:
                        n23 += 1
                for m2 in m[2]:
                    if w2 == m2:
                        n22 += 1
        return  n12, n13, n22, n23

    def doublewin(state, mark):
        win = 0
        for col in valid_moves_list(state):
            next_state = make_move(state,col, mark)
            if connected_four(next_state[mark]):
                next_state_opp = make_move(state, col, 3-mark)
                if connected_four(make_move(next_state_opp, col, mark)[mark]):
                    return True # win with this column first or second move, unstoppable
                win += 1
                if win == 2:
                    return True # more than one winning move
        return False
    
    def count_even_odd(state, mark):
        # helper for heuristic
        # calculate number of marks in even and odd rows
        odd, even = 0, 0
        for row in range(R):
            rd2 = row%2     
            for col in range(C):
                if (state[mark] >> (C*col+row)) & 1:
                    if rd2 == 0:
                        odd += 1
                    else:
                        even += 1
        return even, odd 

    def heuristic(state, mark):
        # the heuristic uses numbers of 2-marks and 3 -marks windows
        # numbers of marks in the even and odd rows
        # and if player or opponent has "doublewin"

        num_twos, num_threes, num_twos_opp, num_threes_opp = num23(state, mark)
        score = 900000*num_threes - 900000*num_threes_opp + 30000*num_twos - 30000*num_twos_opp

        num_evens, num_odds = count_even_odd(state, mark)
        if first: 
            even_odd_rate = num_odds - num_evens
        else:
            even_odd_rate = num_evens - num_odds
        score += 100 * even_odd_rate

        if num_threes > 1:
            if doublewin(state, mark) : score = 1e8
        if num_threes_opp > 1:
            if doublewin(state, 3-mark) : score = -1e8
        return score 

    def score_move(state, col, mark, nsteps):
        next_state = make_move(state, col, mark)
        return minimax(next_state, nsteps-1, -np.Inf, np.Inf, False, mark)
       
    # Minimax implementation
    def minimax(state, depth, a, b, maximizingPlayer, mark):
        if time.time() > FINISH: # timeout
            return 0
        if connected_four(state[mark]): # player win
            return np.inf
        if connected_four(state[3-mark]): # opponent win
            return -np.inf
        if state[1] | state[2] == 279258638311359: # fullboard mask, tie
            return 0

        if depth == 0: # leaf node
            key = (state[1], state[2]) # the key for store state in the dict
            if key in STATES: # heuristic for the state is in tne dict
                return STATES[key] # get it from the dict
            # new state, calculate heuristic
            h = heuristic(state, mark)
            STATES[key] = h # and store it in the dict
            return h
 
        # MiniMax with alpha-beta pruning
    
        valid_moves = valid_moves_list(state)
       
        if maximizingPlayer:
            value = -np.Inf
            for col in valid_moves:
                child = make_move(state, col, mark)
                value = max(value, minimax(child, depth-1, a, b, False, mark))
                if value >= b:
                    break
                a = max(a, value)
            return value
        else:
            value = np.Inf
            for col in valid_moves:
                child = make_move(state, col, 3-mark)
                value = min(value, minimax(child, depth-1, a, b, True, mark))
                if value <= a:
                    break
                b = min(b, value)
            return value 
    
    # The agent begins to work
    
    START = time.time()
    FINISH = START + DELAY
    
    # the dict to store heuristics, a la transposition table
    STATES = {}  
    
    # the depth to start with
    N_STEPS = 6
    
    # Convert the board to a 2D grid and then the grig  to state
    grid = np.asarray(obs.board).reshape(R, C)
    state = grid_to_state(grid)
    
    # the best move for the first move is "3", let's play it
    step = step_number(grid)
    if step < 2:
        return 3
    
    # remember if the agent is the first player, for heuristic
    first = is_first_player(step)

    valid_moves = valid_moves_list(state)
    # check if we have winning move
    for col in valid_moves:
        next_state = make_move(state, col, obs.mark)
        if connected_four(next_state[obs.mark]):
            # yes, we have, let's play it
            return col
        
    # Use the heuristic to assign a score to each possible board in the next step
    # the agent is rather fast, so it starts with six steps to look ahead
    scores = dict(zip(valid_moves, [score_move(state, col, obs.mark, N_STEPS) for col in valid_moves]))

    # Repeat with one step more depth
    while  N_STEPS < 42 - step:
        N_STEPS += 1

        # no need to calculate scores for the moves we already know are the way to loss
        for vm in valid_moves:
            if scores[vm] == -np.inf:
                valid_moves.remove(vm)
        if len(valid_moves) == 0: # we loss anyway, surrender :(
            break
            
        scores_2 = {}
        for col in valid_moves:
            score = score_move(state, col, obs.mark, N_STEPS)
            if score == np.inf:
                # we have the move to win with, let's play it
                return col
            scores_2[col] = score

        if time.time() > FINISH or np.amax(list(scores_2.values())) <= -np.inf:
            # timeout, the search with the next depth isn't complete
            # leave the previous scores as is
            break
        # update the scores withe the result of the next depth search
        scores = scores_2.copy()
    
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]

    # Select the best as nearest to the center of the board from the maximizing columns
    if len(max_cols) >0:
        if 3 in max_cols:
            return 3
        if 2 in max_cols and 4 in max_cols:
            return random.choice([2, 4])
        if 2 in max_cols:
            return 2
        if 4 in max_cols:
            return 4
        
        if 1 in max_cols and 5 in max_cols:
            return random.choice([1, 5])
        if 1 in max_cols:
            return 1
        if 5 in max_cols:
            return 5
        
        if 0 in max_cols and 6 in max_cols:
            return random.choice([0, 6])
        if 0 in max_cols:
            return 0
        if 6 in max_cols:
            return 6
        
        col = random.choice(max_cols)
        return col
    col =  random.choice(max_cols)
    return col 

# Create a submission file

The next code cell writes your agent to a Python file that can be submitted to the competition.

In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "submission.py")

# Validate your submission file

The code cell below has the agent in your submission file play one game round against itself.

If it returns "Success!", then you have correctly defined your agent.

In [ ]:
import sys
from kaggle_environments import utils

out = sys.stdout
submission = utils.read_file("/kaggle/working/submission.py")
agent = utils.get_last_callable(submission)
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")